In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem.cistem import Cistem
import string
import re
from textstat.textstat import *
import seaborn
from textblob_de import TextBlobDE as TextBlob
import autosklearn.classification
import sklearn.metrics
import random
import os
import warnings
import matplotlib.pyplot as plt
from sklearn.utils import resample
%matplotlib inline

/home/mackenzie/anaconda3/lib/python3.7/site-packages/pyparsing.py:2725: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/mackenzie/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# Raw data
train_data = pd.read_csv("/home/mackenzie/Downloads/GermanTrainingData.txt", sep='\t', names=['tweet', 'coarse', 'labels'])
test_data = pd.read_csv("/home/mackenzie/Downloads/GermanTestingData.txt", sep='\t', names=['tweet', 'coarse', 'labels'])
df = pd.concat([train_data, test_data], ignore_index=True)
del train_data
del test_data

In [3]:
tweets=df.tweet

In [4]:
stopwords=stopwords = nltk.corpus.stopwords.words("german")

other_exclusions = ["lbr"]
stopwords.extend(other_exclusions)

stemmer = Cistem()

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-ßA-ß]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-ßA-ß]+", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-ßA-ß.,!?]+", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords,
    use_idf=True,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=3000,
    min_df=5,
    max_df=0.75
    )

In [5]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
#Construct tfidf matrix and get relevant scores
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

/home/mackenzie/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['all', 'ber', 'dami', 'dasselb', 'demselb', 'denselb', 'derselb', 'dess', 'desselb', 'dieselb', 'dor', 'etwa', 'eur', 'f', 'geg', 'hatt', 'hrend', 'jed', 'jen', 'jetz', 'k', 'mach', 'manch', 'nich', 'nne', 'nnt', 'ohn', 'r', 'rde', 'selb', 'solch', 'son', 'sond', 'w', 'welch', 'werd', 'wes', 'woll', 'zwisch'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [7]:
#Get POS tags for tweets and save as a string
tweet_tags = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)

In [8]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None,
    use_idf=False,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=2000,
    min_df=5,
    max_df=0.75,
    )

In [9]:
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

In [10]:
#Now get other features

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-ßA-ß]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = TextBlob("tweet").sentiment
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words, lang='de_DE')
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet)
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment[0],twitter_objs[2], twitter_objs[1], twitter_objs[0]]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [11]:
other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "sentiment", "num_hashtags", "num_mentions", "num_urls"]

In [12]:
feats = get_feature_array(tweets)

In [13]:
#Now join them all up
M = np.concatenate([tfidf,pos,feats],axis=1)

In [14]:
M.shape

(8407, 3918)

In [15]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

In [16]:
# Changing string labels to numeric labels
def string_to_numeric(x):
    if x == 'OTHER' or x == 'PROFANITY':
        return 0
    if x == 'INSULT':
        return 1
    if x == 'ABUSE':
        return 2
    
X = pd.DataFrame(M, columns=feature_names)

# double check that labels are ints
y = df['labels'].apply(string_to_numeric)
print('make sure string to numeric is working...')
print(y.head())


make sure string to numeric is working...
0    0
1    0
2    0
3    0
4    1
Name: labels, dtype: int64


In [17]:
''' In process of testing this...
# Feature Selection RFE -- TODO: issue with nan
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFE(estimator, step=1) # half of the features are selected
selector = selector.fit(X, y)
selector.support_ 
selector.ranking_
'''

' In process of testing this...\n# Feature Selection RFE -- TODO: issue with nan\nfrom sklearn.feature_selection import RFE\nfrom sklearn.svm import SVR\nestimator = SVR(kernel="linear")\nselector = RFE(estimator, step=1) # half of the features are selected\nselector = selector.fit(X, y)\nselector.support_ \nselector.ranking_\n'

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)
y_test = y_test.to_frame(name='labels')
y_train = y_train.to_frame(name='labels')

In [ ]:
# Resampling Technique setup

df = pd.concat([X_train, y_train], axis=1)

# split up the df data by labels
print('other rows extracted...done ')
other = df.loc[df['labels']==0]
print(len(other))

print('insult rows extracted...done: ')
insult = df.loc[df['labels']==1]
print(len(insult))

print('abuse rows extracted...done ')
abuse = df.loc[df['labels']==2]
print(len(abuse))

# Upsample minority class insult
df_insult_upsampled = resample(insult, 
                                 replace=True,     # sample with replacement
                                 n_samples=4722,    # to match majority class
                                 random_state=123) # reproducible results
print('insult upsampled...done')

# Upsample minority class abuse
df_abuse_upsampled = resample(abuse, 
                                 replace=True,     # sample with replacement
                                 n_samples=3926,    # to match majority class
                                 random_state=123) # reproducible results
print('abuse upsampled...done')

In [ ]:
# For Upsampling, update X and y with new values
# resampling doesn't impact the og dataframes, so we make new X and y with updated values
X_train = pd.DataFrame()
y_train = pd.DataFrame()
X_train = pd.concat([other, insult, df_insult_upsampled, abuse, df_abuse_upsampled])
X_train = X_train.drop(columns="labels")
y_train = pd.concat([other['labels'], insult['labels'], df_insult_upsampled['labels'], abuse['labels'], df_abuse_upsampled['labels']])
print('shape of X: ')
print(X_train.shape)
print('shape of y: ')
print(y_train.shape)

shape of X: 
(16214, 3918)
shape of y: 
(16214,)


In [ ]:
print(y_train.head())

180     0
1420    0
6202    0
3159    0
8084    0
Name: labels, dtype: int64


In [ ]:
print("start automl!")
automl = autosklearn.classification.AutoSklearnClassifier(ml_memory_limit=100000, time_left_for_this_task=7200, per_run_time_limit=360)
automl.fit(X_train, y_train, dataset_name='german_data') # feat_type=classification is another param we might use?

start automl!


/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2019-07-29 12:43:52,607:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:43:52,619:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:43:54,623:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:43:56,634:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:43:59,035:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:44:01,073:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:44:03,081:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:44:05,086:EnsembleBuilder(1):german_data] No models better than random - using Dummy Score!
[WARNING] [2019-07-29 12:44:07,091:EnsembleBuilder(1):german_dat

/home/mackenzie/anaconda3/lib/python3.7/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
Process pynisher function call:
Process pynisher function call:
Traceback (most recent call last):
  File "/home/mackenzie/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/mackenzie/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/mackenzie/anaconda3/lib/python3.7/site-packages/pynisher/limit_function_call.py", line 93, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/home/mackenzie/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/mackenzie/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._

In [ ]:
y_hat = automl.predict(X_test)

In [ ]:
print("Accuracy Score:", sklearn.metrics.accuracy_score(y_test, y_hat))
print("F1 score (weighted average): ")
print(sklearn.metrics.f1_score(y_test, y_hat, average="weighted"))

In [ ]:
automl.sprint_statistics() 

In [ ]:
print("Confusion Matrix: ")
print(sklearn.metrics.confusion_matrix(y_test, y_hat))

print("Recall: ")
confusion_matrix = sklearn.metrics.confusion_matrix(y_test,y_hat)
matrix_proportions = np.zeros((3,3))
for i in range(0,3):
    matrix_proportions[i,:] = confusion_matrix[i,:]/float(confusion_matrix[i,:].sum())
names=['Other','Insult','Abuse']
confusion_df = pd.DataFrame(matrix_proportions, index=names,columns=names)
plt.figure(figsize=(5,5))
seaborn.heatmap(confusion_df,annot=True,annot_kws={"size": 12},cmap='gist_gray_r',cbar=False, square=True,fmt='.2f')
plt.ylabel(r'True categories',fontsize=14)
plt.xlabel(r'Predicted categories',fontsize=14)
plt.tick_params(labelsize=12)

In [ ]:
# Create a new folder to hold statistics for a run
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

# create a random folder number to reference the directory
rand = random.randint(0, 1000)*random.randint(0,10)
directory = '/home/mackenzie/workspace/PycharmProjects/DAADRISE_AbusiveLangProject/TestResults/Test_' + str(rand)
print('The results for this run will be stored in ' + directory)
createFolder(directory)

In [ ]:
# Saves the cv results to a csv in the folder specified
results = automl.cv_results_ #A dict with keys as column headers and values as columns, 
                             #that can be imported into a pandas DataFrame

# dict --> pandas dataframe
cv_results = pd.DataFrame.from_dict(results)
print('CV Results saved to directory')
#print(cv_results.head())
cv_results.to_csv(directory+'/cv_results.csv')

In [ ]:
# Saves the classification report info
print(sklearn.metrics.classification_report(y_test, y_hat))
f= open(directory+'/classification_report.txt',"w+")
f.write(sklearn.metrics.classification_report(y_test, y_hat))
f.close()

In [ ]:
# Saves the model info to the directory specified
print('Model Results partially shown below')
models = automl.get_models_with_weights() # Return a list of the final ensemble found by auto-sklearn.
print(models)

# TODO: don't forget to unhighlight and save the models to csv once you know how many models were made!!
# depending on how many models were made might need to change the below line
#model_info = pd.DataFrame.from_dict([models[0]  ])
                            #models[1],models[2],models[3], models[4], models[5], models[6], models[7], models[8], models[9], models[10], models[11]])

#model_info.to_csv(directory+'/model_info.csv')


In [ ]:
# TODO: figure out how to automatically save the png
#plt.savefig(directory+'/recall.png', transparent=True, dpi=400)
#fig = seaborn.heatmap(confusion_df,annot=True,annot_kws={"size": 12},cmap='gist_gray_r',cbar=False, square=True,fmt='.2f')
#fig.savefig(directory+'/recall.png', tranparent=True, dpi=400)